In [1]:
import requests
import pandas as pd
import domojupyter as domo

url_token = "https://accounts.snapchat.com/login/oauth2/access_token"
payload_token = (
    "grant_type=refresh_token&refresh_token=XXXXXXXX&client_id=XXXXXX"
)
headers_token = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cookie': 'sc-wcid=XXXXXX'
}

response_token = requests.post(url_token, headers=headers_token, data=payload_token)

if response_token.status_code == 200:
    access_token = response_token.json().get("access_token", "")

    urls = {
        "Main": "https://api.snapkit.com/v1/stories/studio/analytics/publisher/XXXXX/stories?fromDate=2024-01-01&toDate=2026-01-01",
        "Spanish": "https://api.snapkit.com/v1/stories/studio/analytics/publisher/XXXXXXXXX/stories?fromDate=2024-01-01&toDate=2026-01-01",
        "French": "https://api.snapkit.com/v1/stories/studio/analytics/publisher/XXXXXXXXXX/stories?fromDate=2024-01-01&toDate=2026-01-01",
        "Arabic": "https://api.snapkit.com/v1/stories/studio/analytics/publisher/XXXXXXXXXX/stories?fromDate=2024-01-01&toDate=2026-01-01"
    }
    
    headers_data = {
        'accept': 'application/json',
        'authorization': f'Bearer {access_token}'
    }
    
    
df_list = []

for channel, url in urls.items():
    response_data = requests.get(url, headers=headers_data)
    
    if response_data.status_code == 200:
        data = response_data.json()
        
        if data and 'GLOBAL' in data:

            df_stories = pd.json_normalize(
                data['GLOBAL'],
                record_path='snapList',
                sep='__',
                meta_prefix='meta_',
                record_prefix='snap_',
                meta=[
                    ['metadata', 'storyId'],
                    ['metadata', 'name'],
                    ['metadata', 'postTime'],
                    ['metadata', 'spotlightEndTime'],
                    ['metadata', 'spotlightDuration'],
                    ['metadata', 'tags'],
                    ['metadata', 'numberOfTiles'],
                    ['metadata', 'attachmentType'],
                    ['metadata', 'duration'],
                    ['audience', 'genderDemographics', 'maleUsers', 'percentage'],
                    ['audience', 'genderDemographics', 'maleUsers', 'count'],
                    ['audience', 'genderDemographics', 'femaleUsers', 'percentage'],
                    ['audience', 'genderDemographics', 'femaleUsers', 'count'],
                    ['audience', 'ageDemographics', 'ageRange13To17Users', 'percentage'],
                    ['audience', 'ageDemographics', 'ageRange13To17Users', 'count']
                ],
                errors='ignore'
            )
            
            df_stories['Snap.Channel'] = channel
            
            df_list.append(df_stories)
        else:
            print(f"No 'GLOBAL' data found for {channel}")
    else:
        print(f"Failed to retrieve data for {channel}. Status code:", response_data.status_code)

if df_list:
    final_df = pd.concat(df_list, ignore_index=True)
    final_df['_BATCH_LAST_RUN_'] = pd.Timestamp.now().date()
    
    print("Success")
    print(final_df)
else:
    print("No data retrieved.")
    
    combined_df = combined_df.rename(columns={
})    

    
domo.write_dataframe(final_df, 'Snapchat | Combined Stories Snap Detail | Output')

A newer version of domojupyter is available. Update domojupyter with the following command:

	conda install -c https://domo-conda-prod.s3.amazonaws.com/domo domojupyter=1.1.32
